In [4]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import time
import logging
from typing import Optional

In [13]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from dotenv import load_dotenv
import os
import time

def test_metricool_login():
    """
    Test logging into Metricool and navigating to planner.
    """
    # Load environment variables
    load_dotenv()
    email = os.getenv('METRICOOL_EMAIL')
    password = os.getenv('METRICOOL_PASSWORD')
    
    if not email or not password:
        print("Error: Please set METRICOOL_EMAIL and METRICOOL_PASSWORD in your .env file")
        return False
    
    # Setup Chrome driver with additional options
    options = webdriver.ChromeOptions()
    options.add_argument('--start-maximized')
    options.add_argument('--disable-blink-features=AutomationControlled')
    options.add_experimental_option('excludeSwitches', ['enable-automation'])
    options.add_experimental_option('useAutomationExtension', False)
    
    driver = webdriver.Chrome(options=options)
    
    try:
        # First, navigate to Metricool homepage
        print("Navigating to Metricool homepage...")
        driver.get("https://metricool.com/")
        
        # Wait for and click the login button
        print("Looking for login button...")
        login_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CLASS_NAME, "cta-blanco-login"))
        )
        login_button.click()
        
        # Wait a moment for the login page to load
        time.sleep(3)
        
        # Print current URL to debug
        print(f"Current URL: {driver.current_url}")
        
        # Now handle the login form
        print("Attempting to enter email...")
        email_input = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.ID, "j_username"))
        )
        email_input.clear()
        email_input.send_keys(email)
        
        # Find and fill password field
        print("Attempting to enter password...")
        password_input = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.ID, "j_password"))
        )
        password_input.clear()
        password_input.send_keys(password)
        
        # Click login button
        print("Clicking login submit button...")
        submit_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.ID, "loginFormSubmit"))
        )
        submit_button.click()
        
        # Wait for dashboard to load and look for planner button
        print("Looking for planner button...")
        planner_button = WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "a[href*='/planner'] .fa-calendar-days"))
        )
        
        print("Clicking planner button...")
        planner_button.click()
        
        # Wait a moment to verify we're on the planner page
        time.sleep(5)
        print(f"Current URL after clicking planner: {driver.current_url}")
        
        if "/planner" in driver.current_url:
            print("Successfully navigated to planner!")
            return True
        else:
            print("Navigation to planner seems to have failed")
            driver.save_screenshot("planner_navigation_failed.png")
            return False
            
    except TimeoutException as e:
        print(f"Timeout error: Element not found - {str(e)}")
        print(f"Current URL at error: {driver.current_url}")
        driver.save_screenshot("timeout_error.png")
        print("Screenshot saved as timeout_error.png")
        return False
    except Exception as e:
        print(f"An error occurred: {str(e)}")
        print(f"Current URL at error: {driver.current_url}")
        driver.save_screenshot("general_error.png")
        print("Screenshot saved as general_error.png")
        return False
    finally:
        print("Closing browser...")
        driver.quit()

if __name__ == "__main__":
    success = test_metricool_login()
    print(f"\nLogin and navigation test {'successful' if success else 'failed'}")

Navigating to Metricool homepage...
Looking for login button...
Current URL: https://app.metricool.com/
Attempting to enter email...
Attempting to enter password...
Clicking login submit button...
Looking for planner button...
Clicking planner button...
Current URL after clicking planner: https://app.metricool.com/planner/calendar?blogId=2419811&userId=1976809
Successfully navigated to planner!
Closing browser...

Login and navigation test successful


In [ ]:

class MetricoolPoster:
    def __init__(self, email: str, password: str):
        """
        Initialize MetricoolPoster with login credentials.
        
        Args:
            email (str): Metricool login email
            password (str): Metricool login password
        """
        self.email = email
        self.password = password
        self.driver = None
        self.logger = self._setup_logging()
        
    def _setup_logging(self) -> logging.Logger:
        """Configure logging for the Metricool automation."""
        logging.basicConfig(
            level=logging.INFO,
            format='%(asctime)s - %(levelname)s - %(message)s',
            handlers=[
                logging.FileHandler('metricool_automation.log'),
                logging.StreamHandler()
            ]
        )
        return logging.getLogger(__name__)
    
    def _setup_driver(self):
        """Set up Chrome WebDriver with appropriate options."""
        options = webdriver.ChromeOptions()
        options.add_argument('--start-maximized')
        # Uncomment below line for headless mode
        # options.add_argument('--headless')
        self.driver = webdriver.Chrome(options=options)
        
    def _wait_and_find_element(self, by: By, value: str, timeout: int = 10) -> Optional[webdriver.remote.webelement.WebElement]:
        """
        Wait for and find an element on the page.
        
        Args:
            by: Selenium By method
            value: Element identifier
            timeout: Maximum time to wait in seconds
            
        Returns:
            WebElement if found, None otherwise
        """
        try:
            element = WebDriverWait(self.driver, timeout).until(
                EC.presence_of_element_located((by, value))
            )
            return element
        except TimeoutException:
            self.logger.error(f"Element not found: {value}")
            return None
            
    def login(self) -> bool:
        """
        Log in to Metricool.
        
        Returns:
            bool: True if login successful, False otherwise
        """
        try:
            self.logger.info("Starting login process")
            self.driver.get("https://app.metricool.com/login")
            
            # Wait for and fill email
            email_input = self._wait_and_find_element(By.NAME, "email")
            if not email_input:
                return False
            email_input.send_keys(self.email)
            
            # Wait for and fill password
            password_input = self._wait_and_find_element(By.NAME, "password")
            if not password_input:
                return False
            password_input.send_keys(self.password)
            
            # Click login button
            login_button = self._wait_and_find_element(By.CSS_SELECTOR, "button[type='submit']")
            if not login_button:
                return False
            login_button.click()
            
            # Wait for dashboard to load
            dashboard = self._wait_and_find_element(By.CLASS_NAME, "dashboard")
            return dashboard is not None
            
        except Exception as e:
            self.logger.error(f"Login failed: {str(e)}")
            return False
            
    def post_tweet(self, tweet_text: str) -> bool:
        """
        Post a tweet to Metricool.
        
        Args:
            tweet_text (str): The tweet content to post
            
        Returns:
            bool: True if posting successful, False otherwise
        """
        try:
            self.logger.info("Starting tweet posting process")
            
            # Navigate to post creation
            post_button = self._wait_and_find_element(By.CSS_SELECTOR, "button[data-testid='create-post-button']")
            if not post_button:
                return False
            post_button.click()
            
            # Wait for and fill tweet text
            tweet_input = self._wait_and_find_element(By.CSS_SELECTOR, "div[contenteditable='true']")
            if not tweet_input:
                return False
            tweet_input.send_keys(tweet_text)
            
            # Select Twitter platform (if needed)
            twitter_checkbox = self._wait_and_find_element(By.CSS_SELECTOR, "input[type='checkbox'][value='twitter']")
            if twitter_checkbox and not twitter_checkbox.is_selected():
                twitter_checkbox.click()
            
            # Click post button
            submit_button = self._wait_and_find_element(By.CSS_SELECTOR, "button[type='submit']")
            if not submit_button:
                return False
            submit_button.click()
            
            # Wait for confirmation
            time.sleep(3)
            return True
            
        except Exception as e:
            self.logger.error(f"Posting tweet failed: {str(e)}")
            return False
            
    def close(self):
        """Close the browser and clean up."""
        if self.driver:
            self.driver.quit()
            
    def __enter__(self):
        """Context manager entry."""
        self._setup_driver()
        return self
        
    def __exit__(self, exc_type, exc_val, exc_tb):
        """Context manager exit."""
        self.close()

def post_to_metricool(email: str, password: str, tweet: str) -> bool:
    """
    Main function to post a tweet to Metricool.
    
    Args:
        email (str): Metricool login email
        password (str): Metricool login password
        tweet (str): Tweet content to post
        
    Returns:
        bool: True if posting successful, False otherwise
    """
    with MetricoolPoster(email, password) as poster:
        if poster.login():
            return poster.post_tweet(tweet)
        return False

# Example usage
if __name__ == "__main__":
    from dotenv import load_dotenv
    import os
    
    load_dotenv()
    
    METRICOOL_EMAIL = os.getenv('METRICOOL_EMAIL')
    METRICOOL_PASSWORD = os.getenv('METRICOOL_PASSWORD')
    
    test_tweet = "¡Hola! Este es un tweet de prueba 🚀 #ITAM"
    
    success = post_to_metricool(METRICOOL_EMAIL, METRICOOL_PASSWORD, test_tweet)
    print(f"Tweet posted successfully: {success}")